In [1]:
import numpy as np
from featuremaker import *
from sklearn.svm import LinearSVC

In [2]:
addr = 'INRIAPerson/train_64x128_H96'
addr_ = 'INRIAPerson/test_64x128_H96'

poshists_train = np.array(makehist_positive_norm(addr))
neghists_train = np.array(makehist_negative('Train'))

poshists_test = np.array(makehist_positive_norm(addr_))
neghists_test = np.array(makehist_negative('Test'))

poshists_train = poshists_train.reshape(poshists_train.shape[0], poshists_train.shape[1])
neghists_train = neghists_train.reshape(neghists_train.shape[0], neghists_train.shape[1])
poshists_test = poshists_test.reshape(poshists_test.shape[0], poshists_test.shape[1])
neghists_test = neghists_test.reshape(neghists_test.shape[0], neghists_test.shape[1])

npos =  poshists_train.shape[0]
nneg =  neghists_train.shape[0]

X_train = np.vstack((poshists_train, neghists_train))

Y_train = np.zeros(npos + nneg)
Y_train[:npos] = 1

In [14]:
npos_ = poshists_test.shape[0]
nneg_ = neghists_test.shape[0]

X_test = np.vstack((poshists_test, neghists_test))
Y_test = np.zeros(npos_ + nneg_)
Y_test[:npos_] = 1

In [6]:
clf = LinearSVC(random_state=0, tol=1e-2)
clf.fit(X_train, Y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=0, tol=0.01, verbose=0)

In [17]:
clf.score(X_test, Y_test)

0.9995794785534062

In [9]:
example = poshists_test[100].reshape(1,3780)

In [10]:
clf.predict(example)

array([1.])

In [11]:
from sklearn.svm import SVC

In [ ]:
clf2 = SVC(kernel='linear', tol=1e-2)
clf2.fit(X_train, Y_train)

In [4]:
hog_image = clf2.support_vectors_[1]

NameError: name 'clf2' is not defined

(3780,)

In [2]:
from skimage import data, exposure
import matplotlib.pyplot as plt

In [3]:
hog_image_rescaled = exposure.rescale_intensity(hog_image.reshape, in_range=(0, 2))

NameError: name 'hog_image' is not defined

In [1]:
plt.imshow(hog_image_rescaled.reshape((42,90)));

NameError: name 'plt' is not defined